# 0. Imports

In [193]:
import pandas as pd
from train import save_training_results_to_df, train_epoch, evaluate, translate, my_check_disk_space, save_model_locally
# from preprocess import get_dataloaders2, split_parallel_corpus_chunked, TranslationIterableDataset, get_dataloader_from_files, train_tokenizer
from preprocess import load_tokenizer, get_dataloader_from_files
from test_model import load_tokenizer, test_model, translate_batch, calculate_bleu, save_results_to_df, translate
import torch
from torch.optim import AdamW
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
from tqdm import tqdm
import mlflow
import mlflow.pytorch
from mlflow.models.signature import infer_signature
# from model import Seq2SeqTransformer
import torch.nn.functional as F
import os
import pandas as pd
import datetime
import math
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from tqdm import tqdm
import sentencepiece as spm
import numpy as np
from nltk.translate.bleu_score import corpus_bleu
import nltk
import gc


In [194]:
# del batch, output, loss
torch.cuda.empty_cache()
gc.collect()

17

# 2. Seq2Seq Model

In [195]:
# class PositionalEncoding(nn.Module):
#     def __init__(self, d_model, max_len=5000):
#         super().__init__()
#         pe = torch.zeros(max_len, d_model)
#         position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
#         div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
#         pe[:, 0::2] = torch.sin(position * div_term)
#         pe[:, 1::2] = torch.cos(position * div_term)
#         pe = pe.unsqueeze(0)
#         self.register_buffer('pe', pe)

#     def forward(self, x):
#         return x + self.pe[:, :x.size(1)]

# class Seq2SeqTransformer(nn.Module):
#     def __init__(self, vocab_size, d_model=512, nhead=8, num_encoder_layers=6,
#                  num_decoder_layers=6, dim_feedforward=2048, dropout=0.1):
#         super().__init__()
        
#         self.d_model = d_model
#         self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=0)
#         self.pos_encoder = PositionalEncoding(d_model)
        
#         self.transformer = nn.Transformer(
#             d_model=d_model,
#             nhead=nhead,
#             num_encoder_layers=num_encoder_layers,
#             num_decoder_layers=num_decoder_layers,
#             dim_feedforward=dim_feedforward,
#             dropout=dropout,
#             batch_first=True
#         )
        
#         self.output_layer = nn.Linear(d_model, vocab_size)
        
#     def generate_square_subsequent_mask(self, sz):
#         mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
#         mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
#         return mask
    
#     def forward(self, src, tgt, src_mask=None, tgt_mask=None):
#         # src: [batch_size, src_len]
#         # tgt: [batch_size, tgt_len]
        
#         # Create target mask for decoder
#         if tgt_mask is None:
#             tgt_mask = self.generate_square_subsequent_mask(tgt.size(1)).to(tgt.device)
        
#         # Embedding and positional encoding
#         src = self.embedding(src) * math.sqrt(self.d_model)
#         src = self.pos_encoder(src)
        
#         tgt = self.embedding(tgt) * math.sqrt(self.d_model)
#         tgt = self.pos_encoder(tgt)
#         # print("tgt max index:", tgt.max())
#         print("tgt shape:", tgt.shape)
#         print("tgt max index:", tgt.max())  # before embedding()
#         print("embedding vocab size:", self.embedding.num_embeddings)

        
#         # Transformer forward pass
#         # output = self.transformer(src, tgt, src_mask, tgt_mask)

#         #new approach ignores padded areas that confused prediction before
#         src_key_padding_mask = (src == self.embedding.padding_idx)
#         tgt_key_padding_mask = (tgt == self.embedding.padding_idx)

#         output = self.transformer(
#             src, tgt,
#             src_mask=src_mask,
#             tgt_mask=tgt_mask,
#             src_key_padding_mask=src_key_padding_mask,
#             tgt_key_padding_mask=tgt_key_padding_mask,
#             memory_key_padding_mask=src_key_padding_mask
#         )

        
#         # Project to vocabulary size
#         output = self.output_layer(output)
        
#         return output
    
#     def encode(self, src):
#         src = self.embedding(src) * math.sqrt(self.d_model)
#         src = self.pos_encoder(src)
#         return self.transformer.encoder(src)
    
#     def decode(self, tgt, memory, tgt_mask=None):
#         if tgt_mask is None:
#             tgt_mask = self.generate_square_subsequent_mask(tgt.size(1)).to(tgt.device)
        
#         tgt = self.embedding(tgt) * math.sqrt(self.d_model)
#         tgt = self.pos_encoder(tgt)
        
#         output = self.transformer.decoder(tgt, memory, tgt_mask)
#         output = self.output_layer(output)
#         return output

In [196]:
# import torch
# import torch.nn as nn
# import math

# class PositionalEncoding(nn.Module):
#     def __init__(self, d_model, max_len=5000):
#         super().__init__()
#         pe = torch.zeros(max_len, d_model)
#         position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
#         div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
#         pe[:, 0::2] = torch.sin(position * div_term)
#         pe[:, 1::2] = torch.cos(position * div_term)
#         pe = pe.unsqueeze(0)
#         self.register_buffer('pe', pe)

#     def forward(self, x):
#         return x + self.pe[:, :x.size(1)]

# class Seq2SeqTransformer(nn.Module):
#     def __init__(self, vocab_size, d_model=512, nhead=8, num_encoder_layers=6,
#                  num_decoder_layers=6, dim_feedforward=2048, dropout=0.1):
#         super().__init__()
        
#         self.d_model = d_model
#         self.embedding = nn.Embedding(vocab_size, d_model)
#         self.pos_encoder = PositionalEncoding(d_model)
        
#         self.transformer = nn.Transformer(
#             d_model=d_model,
#             nhead=nhead,
#             num_encoder_layers=num_encoder_layers,
#             num_decoder_layers=num_decoder_layers,
#             dim_feedforward=dim_feedforward,
#             dropout=dropout,
#             batch_first=True
#         )
        
#         self.output_layer = nn.Linear(d_model, vocab_size)
        
#     def generate_square_subsequent_mask(self, sz):
#         mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
#         mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
#         return mask
    
#     def forward(self, src, tgt, src_mask=None, tgt_mask=None):
#         # src: [batch_size, src_len]
#         # tgt: [batch_size, tgt_len]
        
#         # Create target mask for decoder
#         if tgt_mask is None:
#             tgt_mask = self.generate_square_subsequent_mask(tgt.size(1)).to(tgt.device)
        
#         # Create source padding mask
#         src_padding_mask = (src == 0).to(src.device)  # True for padding tokens
#         tgt_padding_mask = (tgt == 0).to(tgt.device)  # True for padding tokens
        
#         # Embedding and positional encoding
#         src = self.embedding(src) * math.sqrt(self.d_model)
#         src = self.pos_encoder(src)
        
#         tgt = self.embedding(tgt) * math.sqrt(self.d_model)
#         tgt = self.pos_encoder(tgt)
#         print("tgt max index:", tgt.max())
#         print("tgt shape:", tgt.shape)
#         print("embedding vocab size:", self.embedding.num_embeddings)

        
#         # Transformer forward pass
#         output = self.transformer(
#             src, 
#             tgt,
#             src_mask=None,  # No source mask needed
#             tgt_mask=tgt_mask,
#             src_key_padding_mask=src_padding_mask,
#             tgt_key_padding_mask=tgt_padding_mask,
#             memory_key_padding_mask=src_padding_mask
#         )
        
#         # Project to vocabulary size
#         output = self.output_layer(output)
        
#         return output
    
#     def encode(self, src):
#         src = self.embedding(src) * math.sqrt(self.d_model)
#         src = self.pos_encoder(src)
#         return self.transformer.encoder(src)
    
#     def decode(self, tgt, memory, tgt_mask=None):
#         if tgt_mask is None:
#             tgt_mask = self.generate_square_subsequent_mask(tgt.size(1)).to(tgt.device)
        
#         tgt = self.embedding(tgt) * math.sqrt(self.d_model)
#         tgt = self.pos_encoder(tgt)
        
#         output = self.transformer.decoder(tgt, memory, tgt_mask)
#         output = self.output_layer(output)
#         return output 

In [197]:
# import torch
# import torch.nn as nn
# import math

# class PositionalEncoding(nn.Module):
#     def __init__(self, d_model, max_len=5000):
#         super().__init__()
#         pe = torch.zeros(max_len, d_model)
#         position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
#         div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
#         pe[:, 0::2] = torch.sin(position * div_term)
#         pe[:, 1::2] = torch.cos(position * div_term)
#         pe = pe.unsqueeze(0)
#         self.register_buffer('pe', pe)

#     def forward(self, x):
#         return x + self.pe[:, :x.size(1)]

# class Seq2SeqTransformer(nn.Module):
#     def __init__(self, vocab_size, d_model=512, nhead=8, num_encoder_layers=6,
#                  num_decoder_layers=6, dim_feedforward=2048, dropout=0.1):
#         super().__init__()
        
#         self.d_model = d_model
#         self.embedding = nn.Embedding(vocab_size, d_model)
#         self.pos_encoder = PositionalEncoding(d_model)
        
#         self.transformer = nn.Transformer(
#             d_model=d_model,
#             nhead=nhead,
#             num_encoder_layers=num_encoder_layers,
#             num_decoder_layers=num_decoder_layers,
#             dim_feedforward=dim_feedforward,
#             dropout=dropout,
#             batch_first=True
#         )
        
#         self.output_layer = nn.Linear(d_model, vocab_size)
        
#     def generate_square_subsequent_mask(self, sz):
#         mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
#         mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
#         return mask
    
#     def forward(self, src, tgt, src_mask=None, tgt_mask=None):
#         # src: [batch_size, src_len]
#         # tgt: [batch_size, tgt_len]
        
#         try:
#             # Create target mask for decoder
#             if tgt_mask is None:
#                 tgt_mask = self.generate_square_subsequent_mask(tgt.size(1)).to(tgt.device)
            
#             # Create source padding mask
#             src_padding_mask = (src == 0).to(src.device)  # True for padding tokens
#             tgt_padding_mask = (tgt == 0).to(tgt.device)  # True for padding tokens
            
#             # Embedding and positional encoding
#             src = self.embedding(src) * math.sqrt(self.d_model)
#             src = self.pos_encoder(src)
            
#             tgt = self.embedding(tgt) * math.sqrt(self.d_model)
#             tgt = self.pos_encoder(tgt)
            
#             # Transformer forward pass
#             output = self.transformer(
#                 src, 
#                 tgt,
#                 src_mask=None,  # No source mask needed
#                 tgt_mask=tgt_mask,
#                 src_key_padding_mask=src_padding_mask,
#                 tgt_key_padding_mask=tgt_padding_mask,
#                 memory_key_padding_mask=src_padding_mask
#             )
            
#             # Project to vocabulary size
#             output = self.output_layer(output)
            
#             return output
            
#         except Exception as e:
#             print(f"Error in forward pass: {str(e)}")
#             print(f"Input shapes - src: {src.shape}, tgt: {tgt.shape}")
#             print(f"Device - src: {src.device}, tgt: {tgt.device}")
#             raise
    
#     def encode(self, src):
#         src = self.embedding(src) * math.sqrt(self.d_model)
#         src = self.pos_encoder(src)
#         return self.transformer.encoder(src)
    
#     def decode(self, tgt, memory, tgt_mask=None):
#         if tgt_mask is None:
#             tgt_mask = self.generate_square_subsequent_mask(tgt.size(1)).to(tgt.device)
        
#         tgt = self.embedding(tgt) * math.sqrt(self.d_model)
#         tgt = self.pos_encoder(tgt)
        
#         output = self.transformer.decoder(tgt, memory, tgt_mask)
#         output = self.output_layer(output)
#         return output 

In [198]:
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        print("Model embedding vocab size:", self.embedding.num_embeddings)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

class Seq2SeqTransformer(nn.Module):
    def __init__(self, vocab_size, d_model=512, nhead=8, num_encoder_layers=6,
                 num_decoder_layers=6, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        
        self.d_model = d_model
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        
        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True
        )
        
        self.output_layer = nn.Linear(d_model, vocab_size)

        print("Embedding matrix shape:", self.embedding.weight.shape)

        
    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask
    
    # def forward(self, src, tgt, src_mask=None, tgt_mask=None):
    #     # src: [batch_size, src_len]
    #     # tgt: [batch_size, tgt_len]
        
    #     try:
    #         # Create target mask for decoder
    #         if tgt_mask is None:
    #             tgt_mask = self.generate_square_subsequent_mask(tgt.size(1)).to(tgt.device)
            
    #         # Create source padding mask
    #         src_padding_mask = (src == 0).to(src.device)  # True for padding tokens
    #         tgt_padding_mask = (tgt == 0).to(tgt.device)  # True for padding tokens
            
    #         # Embedding and positional encoding
    #         src = self.embedding(src) * math.sqrt(self.d_model)
    #         src = self.pos_encoder(src)
            
    #         tgt = self.embedding(tgt) * math.sqrt(self.d_model)
    #         tgt = self.pos_encoder(tgt)
            
    #         # Transformer forward pass
    #         output = self.transformer(
    #             src, 
    #             tgt,
    #             src_mask=None,  # No source mask needed
    #             tgt_mask=tgt_mask,
    #             src_key_padding_mask=src_padding_mask,
    #             tgt_key_padding_mask=tgt_padding_mask,
    #             memory_key_padding_mask=src_padding_mask
    #         )
            
    #         # Project to vocabulary size
    #         output = self.output_layer(output)
            
    #         return output
            
    #     except Exception as e:
    #         print(f"Error in forward pass: {str(e)}")
    #         print(f"Input shapes - src: {src.shape}, tgt: {tgt.shape}")
    #         print(f"Device - src: {src.device}, tgt: {tgt.device}")
    #         raise
    
    # def encode(self, src):
    #     src = self.embedding(src) * math.sqrt(self.d_model)
    #     src = self.pos_encoder(src)
    #     return self.transformer.encoder(src)
    
    # def decode(self, tgt, memory, tgt_mask=None):
    #     if tgt_mask is None:
    #         tgt_mask = self.generate_square_subsequent_mask(tgt.size(1)).to(tgt.device)
        
    #     tgt = self.embedding(tgt) * math.sqrt(self.d_model)
    #     tgt = self.pos_encoder(tgt)
        
    #     output = self.transformer.decoder(tgt, memory, tgt_mask)
    #     output = self.output_layer(output)
    #     return output 
    def forward(self, src, tgt, src_mask=None, tgt_mask=None, src_key_padding_mask=None, tgt_key_padding_mask=None):
    # Create subsequent mask for decoder if not provided
        if tgt_mask is None:
            tgt_seq_len = tgt.size(1)
            tgt_mask = self.generate_square_subsequent_mask(tgt_seq_len).to(tgt.device).bool()

        if src_key_padding_mask is None:
            src_key_padding_mask = (src == 0)  # assuming pad_id = 0
        if tgt_key_padding_mask is None:
            tgt_key_padding_mask = (tgt == 0)

        # Embed and positionally encode
        src = self.embedding(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)

        tgt = self.embedding(tgt) * math.sqrt(self.d_model)
        tgt = self.pos_encoder(tgt)

        # Transformer
        output = self.transformer(
            src,
            tgt,
            tgt_mask=tgt_mask,
            src_key_padding_mask=src_key_padding_mask.bool(),
            tgt_key_padding_mask=tgt_key_padding_mask.bool()
        )

        # Output projection
        return self.output_layer(output)


# 3. Model Training

In [199]:
# try:
#     #     # Set up MLflow
#     # mlflow.set_tracking_uri("file:./mlruns")
#     # mlflow.set_experiment("german-english-translator")
    
#     # Set device
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     print(f"Using device: {device}")
    
#     # Get dataloaders and tokenizer
#     train_loader, val_loader, test_loader, tokenizer = get_dataloaders(
#         'de-en/europarl-v7.de-en.de',
#         'de-en/europarl-v7.de-en.en',
#         batch_size=32
#     )
#     # print("pad_token_id:", tokenizer.pad_id() if hasattr(tokenizer, "pad_id") else "n/a")
#     # print("unk_token_id:", tokenizer.unk_id() if hasattr(tokenizer, "unk_id") else "n/a")
#     pad_id = tokenizer.pad_id()
    
#     # Model parameters
#     model_params = {
#         "vocab_size": tokenizer.get_piece_size(),
#         "d_model": 512,
#         "nhead": 8,
#         "num_encoder_layers": 6,
#         "num_decoder_layers": 6,
#         "dim_feedforward": 2048,
#         "dropout": 0.1
#     }
    
#     # Training parameters
#     training_params = {
#         "learning_rate": 1e-4,
#         "betas" : (0.9, 0.98), # typical for tranformer training
#         "weight_decay":0.01, # L2 reg
#         "num_epochs": 10,
#         "batch_size": 32
#     }

#     try:
#         # with mlflow.start_run() as run:
#         #     print("✅ Entered MLflow run")
#         #     print("Run ID:", run.info.run_id)
#         #     run_id = run.info.run_id
#         #     mlflow.log_param("debug_check", 1)  # confirm it's logging
#         #     # Log parameters
#         #     mlflow.log_params({**model_params, **training_params})
            
#             # Initialize model
#         model = Seq2SeqTransformer(**model_params).to(device)
        
#         # Initialize optimizer
#         optimizer = AdamW(model.parameters(), lr=training_params["learning_rate"],\
#                         betas = training_params["betas"],\
#                             weight_decay = training_params["weight_decay"])
        
#         # Training loop
#         best_val_loss = float('inf')
#         training_history = []
        
#         for epoch in range(training_params["num_epochs"]):
#             print(f"\nEpoch {epoch + 1}/{training_params['num_epochs']}")
            
#             # Train
#             train_loss = train_epoch(model, train_loader, optimizer, device, pad_id)
            
#             # Evaluate
#             val_loss = evaluate(model, val_loader, device)
            
#             # Log metrics
#             mlflow.log_metrics({
#                 "train_loss": train_loss,
#                 "val_loss": val_loss,
#                 "epoch": epoch + 1
#             }, step=epoch)
            
#             print(f"Train Loss: {train_loss:.4f}")
#             print(f"Val Loss: {val_loss:.4f}")
            
#             # Test translation
#             print("\n" + "="*50)
#             print("SAMPLE PREDICTION")
#             print("="*50)
#             test_text = "Hallo, wie geht es dir?"
#             translation = translate(model, tokenizer, test_text, device)
#             print(f"German: {test_text}")
#             print(f"English: {translation}")
#             print("="*50 + "\n")
            
#             # Save epoch results
#             epoch_results = {
#                 'epoch': epoch + 1,
#                 'train_loss': train_loss,
#                 'val_loss': val_loss,
#                 'model_path': f"mlruns/{run.info.experiment_id}/{run.info.run_id}/artifacts/model_epoch_{epoch + 1}",
#                 'batch_size': training_params['batch_size'],
#                 'learning_rate': training_params['learning_rate'],
#                 'device': str(device),
#                 'test_translation_source': test_text,
#                 'test_translation_output': translation
#             }
#             training_history.append(epoch_results)
            
#             # Log model checkpoint
#             checkpoint_path = f"checkpoints/epoch_{epoch + 1}"
#             os.makedirs("checkpoints", exist_ok=True)
#             torch.save({
#                 'epoch': epoch + 1,
#                 'model_state_dict': model.state_dict(),
#                 'optimizer_state_dict': optimizer.state_dict(),
#                 'train_loss': train_loss,
#                 'val_loss': val_loss,
#             }, f"{checkpoint_path}.pt")
                
#             # # Log model to MLflow
#             # mlflow.pytorch.log_model(model, f"model_epoch_{epoch + 1}")
            
#             # # Log sample translation
#             # mlflow.log_text(translation, f"translations/epoch_{epoch + 1}.txt")
            
#             # # Get signatures
#             # example_input = torch.randint(0, tokenizer.get_piece_size(), (1, 12)).to(device)
#             # example_target = torch.randint(0, tokenizer.get_piece_size(), (1, 11)).to(device)
#             # input_example = (example_input, example_target)

#             # # Run one inference pass to get output for signature
#             # example_output = model(example_input, example_target)
#             # signature = infer_signature(
#             #     inputs={"input_ids": example_input.cpu().numpy(), "target_input": example_target.cpu().numpy()},
#             #     outputs=example_output.detach().cpu().numpy()
#             # )
            
#             # # Save final model
#             # mlflow.pytorch.log_model(model, "final_model", input_example=input_example, signature=signature)
            
#             # Save all training results to DataFrame
#             df = pd.DataFrame(training_history)
#             timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
#             results_file = f"training_results_{timestamp}.csv"
#             df.to_csv(results_file, index=False)
#             print(f"\nDEBUG: Saved training results to {os.path.abspath(results_file)}")
            
#             # Log the DataFrame as an artifact
#             os.makedirs("training_results", exist_ok=True)
#             metrics_file = f"training_results/metrics_{timestamp}.csv"
#             df.to_csv(metrics_file, index=False)
#             print(f"DEBUG: Saved metrics to {os.path.abspath(metrics_file)}")
#             mlflow.log_artifact("training_results")
#             print("DEBUG: Logged artifacts to MLflow")
            
#             # Print the model path for the shell script to capture
#             # model_path = f"mlruns/{run.info.experiment_id}/{run.info.run_id}/artifacts/final_model"
#             # print(f"\nDEBUG: Final model path: {os.path.abspath(model_path)}")
#             # print(f"MODEL_PATH={model_path}")

#     except Exception as e:
#         print("❌ MLflow run failed:", e)
#         print("DEBUG: Exception details:", str(e))
#         import traceback
#         print("DEBUG: Full traceback:")
#         print(traceback.format_exc())

# except Exception as e:
#         print("❌ Main function failed:", e)
#         print("DEBUG: Exception details:", str(e))
#         import traceback
#         print("DEBUG: Full traceback:")
#         print(traceback.format_exc())

In [200]:
print("hello world")

hello world


In [201]:
# print("Training now...")
# # Set device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")



# split_parallel_corpus_chunked(
#     de_file='de-en/europarl-v7.de-en.de',
#     en_file='de-en/europarl-v7.de-en.en',
#     output_dir='de-en/split',
#     train_ratio=0.7,
#     val_ratio=0.2,
#     test_ratio=0.1,
#     lim=10000  # or None for full dataset
# )


# train_loader, val_loader, test_loader, tokenizer = get_dataloaders2(
#     train_de='de-en/split/train.de',
#     train_en='de-en/split/train.en',
#     val_de='de-en/split/val.de',
#     val_en='de-en/split/val.en',
#     test_de='de-en/split/test.de',
#     test_en='de-en/split/test.en',
#     batch_size=8,
#     vocab_train_src='de-en/europarl-v7.de-en.de',
#     vocab_train_tgt='de-en/europarl-v7.de-en.en'
# )


# # Get dataloaders and tokenizer
# # train_loader, val_loader, test_loader, tokenizer = get_dataloaders(
# #     'de-en/europarl-v7.de-en.de',
# #     'de-en/europarl-v7.de-en.en',
# #     batch_size=32,
# #     lim=10000
# # )


# # print("pad_token_id:", tokenizer.pad_id() if hasattr(tokenizer, "pad_id") else "n/a")
# # print("unk_token_id:", tokenizer.unk_id() if hasattr(tokenizer, "unk_id") else "n/a")
# pad_id = tokenizer.pad_id()

# # Model parameters, halfing these since way less data now
# model_params = {
#     "vocab_size": tokenizer.get_piece_size(),
#     "d_model": 256,
#     "nhead": 4,
#     "num_encoder_layers": 3,
#     "num_decoder_layers": 3,
#     "dim_feedforward": 1024,
#     "dropout": 0.2
# }

# # Training parameters
# training_params = {
#     "learning_rate": 5e-5,
#     "betas": (0.9, 0.98),
#     "weight_decay": 0.1,
#     "num_epochs": 10,
#     "batch_size": 32,
#     "warmup_steps": 1000,
#     "gradient_clip_val": 1.0,
#     "dropout": 0.2
# }

# try:
#     # Initialize model and optimizer
#     model = Seq2SeqTransformer(**model_params).to(device)
#     optimizer = AdamW(model.parameters(), 
#                     lr=training_params["learning_rate"],
#                     betas=training_params["betas"],
#                     weight_decay=training_params["weight_decay"])
    
#     # Add learning rate scheduler
#     def get_lr_scheduler(optimizer, warmup_steps):
#         def lr_lambda(step):
#             if step < warmup_steps:
#                 return float(step) / float(max(1, warmup_steps))
#             return 1.0
#         return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

#     scheduler = get_lr_scheduler(optimizer, training_params["warmup_steps"])
#     global_step = 0
    
#     # Training loop
#     best_val_loss = float('inf')
#     training_history = []
    
#     for epoch in range(training_params["num_epochs"]):
#         print(f"\nEpoch {epoch + 1}/{training_params['num_epochs']}")
        
#         # Train
#         train_loss = train_epoch(model, train_loader, optimizer, device, pad_id)
        
#         # Evaluate
#         val_loss = evaluate(model, val_loader, device)
        
#         # Save checkpoint locally (keeping only last 2)
#         checkpoint_path = save_model_locally(model, optimizer, epoch + 1, train_loss, val_loss)
#         print(f"Saved checkpoint to {checkpoint_path}")
        
#         # Save epoch results
#         epoch_results = {
#             'epoch': epoch + 1,
#             'train_loss': train_loss,
#             'val_loss': val_loss,
#             'checkpoint_path': checkpoint_path,
#             'batch_size': training_params['batch_size'],
#             'learning_rate': training_params['learning_rate'],
#             'device': str(device)
#         }
#         training_history.append(epoch_results)
        
#         # Save results to CSV
#         df = pd.DataFrame(training_history)
#         results_file = f"training_results_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
#         df.to_csv(results_file, index=False)
#         print(f"Saved training results to {results_file}")
        
#         # Update learning rate
#         scheduler.step(global_step)
#         global_step += 1
        
#         print(f"Train Loss: {train_loss:.4f}")
#         print(f"Val Loss: {val_loss:.4f}")
        
#         # Test translation
#         test_text = "Hallo, wie geht es dir?"
#         translation = translate(model, tokenizer, test_text, device)
#         print(f"\nSample translation:")
#         print(f"German: {test_text}")
#         print(f"English: {translation}")
        
#         # Check disk space before next epoch
#         if not my_check_disk_space():
#             print("Warning: Low disk space. Saving current state and stopping training.")
#             break

# except Exception as e:
#     print(f"Error during training: {str(e)}")
#     import traceback
#     print(traceback.format_exc())
#     # Save the model state even if training fails
#     if 'model' in locals():
#         save_model_locally(model, optimizer, epoch + 1, train_loss, val_loss, path="error_checkpoints")
#     sys.exit(1)

In [202]:
# print("Training now...")
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# batch_size = 32
# tokenizer = train_tokenizer("de-en/europarl-v7.de-en.de", "de-en/europarl-v7.de-en.en")

# split_parallel_corpus_chunked(
#     de_file='de-en/europarl-v7.de-en.de',
#     en_file='de-en/europarl-v7.de-en.en',
#     output_dir='de-en/split',
#     train_ratio=0.7,
#     val_ratio=0.2,
#     test_ratio=0.1,
#     lim=10000  # or None for full dataset
# )

# train_loader = get_dataloader_from_files("train.de", "train.en", tokenizer, batch_size)
# val_loader = get_dataloader_from_files("val.de", "val.en", tokenizer, batch_size)
# test_loader = get_dataloader_from_files("test.de", "test.en", tokenizer, batch_size)

# pad_id = tokenizer.pad_id()

# # Model parameters, halfing these since way less data now
# model_params = {
#     "vocab_size": tokenizer.get_piece_size(),
#     "d_model": 256,
#     "nhead": 4,
#     "num_encoder_layers": 3,
#     "num_decoder_layers": 3,
#     "dim_feedforward": 1024,
#     "dropout": 0.2
# }

# # Training parameters
# training_params = {
#     "learning_rate": 5e-5,
#     "betas": (0.9, 0.98),
#     "weight_decay": 0.1,
#     "num_epochs": 10,
#     "batch_size": 32,
#     "warmup_steps": 1000,
#     "gradient_clip_val": 1.0,
#     "dropout": 0.2
# }

# try:
#     # Initialize model and optimizer
#     model = Seq2SeqTransformer(**model_params).to(device)
#     optimizer = AdamW(model.parameters(), 
#                     lr=training_params["learning_rate"],
#                     betas=training_params["betas"],
#                     weight_decay=training_params["weight_decay"])
    
#     # Add learning rate scheduler
#     def get_lr_scheduler(optimizer, warmup_steps):
#         def lr_lambda(step):
#             if step < warmup_steps:
#                 return float(step) / float(max(1, warmup_steps))
#             return 1.0
#         return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

#     scheduler = get_lr_scheduler(optimizer, training_params["warmup_steps"])
#     global_step = 0
    
#     # Training loop
#     best_val_loss = float('inf')
#     training_history = []
    
#     for epoch in range(training_params["num_epochs"]):
#         print(f"\nEpoch {epoch + 1}/{training_params['num_epochs']}")
        
#         # Train
#         train_loss = train_epoch(model, train_loader, optimizer, device, pad_id)
        
#         # Evaluate
#         val_loss = evaluate(model, val_loader, device)
        
#         # Save checkpoint locally (keeping only last 2)
#         checkpoint_path = save_model_locally(model, optimizer, epoch + 1, train_loss, val_loss)
#         print(f"Saved checkpoint to {checkpoint_path}")
        
#         # Save epoch results
#         epoch_results = {
#             'epoch': epoch + 1,
#             'train_loss': train_loss,
#             'val_loss': val_loss,
#             'checkpoint_path': checkpoint_path,
#             'batch_size': training_params['batch_size'],
#             'learning_rate': training_params['learning_rate'],
#             'device': str(device)
#         }
#         training_history.append(epoch_results)
        
#         # Save results to CSV
#         df = pd.DataFrame(training_history)
#         results_file = f"training_results_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
#         df.to_csv(results_file, index=False)
#         print(f"Saved training results to {results_file}")
        
#         # Update learning rate
#         scheduler.step(global_step)
#         global_step += 1
        
#         print(f"Train Loss: {train_loss:.4f}")
#         print(f"Val Loss: {val_loss:.4f}")
        
#         # Test translation
#         test_text = "Hallo, wie geht es dir?"
#         translation = translate(model, tokenizer, test_text, device)
#         print(f"\nSample translation:")
#         print(f"German: {test_text}")
#         print(f"English: {translation}")
        
#         # Check disk space before next epoch
#         if not my_check_disk_space():
#             print("Warning: Low disk space. Saving current state and stopping training.")
#             break

# except Exception as e:
#     print(f"Error during training: {str(e)}")
#     import traceback
#     print(traceback.format_exc())
#     # Save the model state even if training fails
#     if 'model' in locals():
#         save_model_locally(model, optimizer, epoch + 1, train_loss, val_loss, path="error_checkpoints")
#     sys.exit(1)

In [203]:
# print("Training now...")
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")
# batch_size = 32

# # ✅ 1. Run this ONCE — then comment it out for all future runs
# if not os.path.exists("translation_tokenizer.model"):
#     train_tokenizer("de-en/europarl-v7.de-en.de", "de-en/europarl-v7.de-en.en")

# if not os.path.exists("de-en/split/train.de"):
#     split_parallel_corpus_chunked(
#         de_file='de-en/europarl-v7.de-en.de',
#         en_file='de-en/europarl-v7.de-en.en',
#         output_dir='de-en/split',
#         train_ratio=0.7,
#         val_ratio=0.2,
#         test_ratio=0.1,
#         lim=10000  # ✅ limit before any tokenizing happens
#     )

# # ✅ 2. Load tokenizer from file (no data loaded into memory)
# tokenizer = load_tokenizer()

# # ✅ 3. Use only pre-split, line-limited files
# train_loader = get_dataloader_from_files("de-en/split/train.de", "de-en/split/train.en", tokenizer, batch_size)
# val_loader = get_dataloader_from_files("de-en/split/val.de", "de-en/split/val.en", tokenizer, batch_size)
# test_loader = get_dataloader_from_files("de-en/split/test.de", "de-en/split/test.en", tokenizer, batch_size)

# pad_id = tokenizer.pad_id()


# # Model parameters, halfing these since way less data now
# model_params = {
#     "vocab_size": tokenizer.get_piece_size(),
#     "d_model": 256,
#     "nhead": 4,
#     "num_encoder_layers": 3,
#     "num_decoder_layers": 3,
#     "dim_feedforward": 1024,
#     "dropout": 0.2
# }

# # Training parameters
# training_params = {
#     "learning_rate": 5e-5,
#     "betas": (0.9, 0.98),
#     "weight_decay": 0.1,
#     "num_epochs": 10,
#     "batch_size": 32,
#     "warmup_steps": 1000,
#     "gradient_clip_val": 1.0,
#     "dropout": 0.2
# }

# try:
#     # Initialize model and optimizer
#     model = Seq2SeqTransformer(**model_params).to(device)
#     optimizer = AdamW(model.parameters(), 
#                     lr=training_params["learning_rate"],
#                     betas=training_params["betas"],
#                     weight_decay=training_params["weight_decay"])
    
#     # Add learning rate scheduler
#     def get_lr_scheduler(optimizer, warmup_steps):
#         def lr_lambda(step):
#             if step < warmup_steps:
#                 return float(step) / float(max(1, warmup_steps))
#             return 1.0
#         return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

#     scheduler = get_lr_scheduler(optimizer, training_params["warmup_steps"])
#     global_step = 0
    
#     # Training loop
#     best_val_loss = float('inf')
#     training_history = []
    
#     for epoch in range(training_params["num_epochs"]):
#         print(f"\nEpoch {epoch + 1}/{training_params['num_epochs']}")
        
#         # Train
#         train_loss = train_epoch(model, train_loader, optimizer, device, pad_id)
        
#         # Evaluate
#         val_loss = float('inf')
#         try:
#             val_loss = evaluate(model, val_loader, device)
#         except Exception as e:
#             print(f"Validation failed: {str(e)}")
        
#         # Save checkpoint locally (keeping only last 2)
#         # val_loss = val_loss if 'val_loss' in locals() else None
#         checkpoint_path = save_model_locally(model, optimizer, epoch + 1, train_loss, val_loss, path="error_checkpoints")
#         print(f"Saved checkpoint to {checkpoint_path}")
        
#         # Save epoch results
#         epoch_results = {
#             'epoch': epoch + 1,
#             'train_loss': train_loss,
#             'val_loss': val_loss,
#             'checkpoint_path': checkpoint_path,
#             'batch_size': training_params['batch_size'],
#             'learning_rate': training_params['learning_rate'],
#             'device': str(device)
#         }
#         training_history.append(epoch_results)
        
#         # Save results to CSV
#         df = pd.DataFrame(training_history)
#         results_file = f"training_results_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
#         df.to_csv(results_file, index=False)
#         print(f"Saved training results to {results_file}")
        
#         # Update learning rate
#         scheduler.step()
#         global_step += 1
        
#         print(f"Train Loss: {train_loss:.4f}")
#         print(f"Val Loss: {val_loss:.4f}")
        
#         # Test translation
#         test_text = "Hallo, wie geht es dir?"
#         translation = translate(model, tokenizer, test_text, device)
#         print(f"\nSample translation:")
#         print(f"German: {test_text}")
#         print(f"English: {translation}")
        
#         # Check disk space before next epoch
#         if not my_check_disk_space():
#             print("Warning: Low disk space. Saving current state and stopping training.")
#             break

# except Exception as e:
#     print(f"Error during training: {str(e)}")
#     import traceback
#     print(traceback.format_exc())
#     # Save the model state even if training fails
#     if 'model' in locals():
#         save_model_locally(model, optimizer, epoch + 1, train_loss, val_loss, path="error_checkpoints")
#     sys.exit(1)

In [204]:
!python train.py

Using device: cpu
Training new tokenizer...
Train: 8000, Val: 1000, Test: 1000
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: tokenizer_input.txt
  input_format: 
  model_prefix: translation_tokenizer
  model_type: BPE
  vocab_size: 32000
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  user_defined_symbols: [DE]
  user_defined_symbols: [EN]
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 1
  bos_id: 2
  eos_id

# 4. Model Testing